
#  Game Analysis

##  Heroes of Pymoli
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

### Setup

In [1]:
#  Dependencies
import pandas as pd
import numpy as np


In [2]:
#  Get source data for analysis
sourcefile = "Resources/purchase_data.csv"
raw_csv = pd.read_csv(sourcefile)

#  Preview source data
raw_csv.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


### Player Count

In [3]:
#  Identify the number of purchases made by each player who engaged in a transaction
player_purchases = raw_csv["SN"].value_counts()

#  Identify total number of players who engaged in a transaction
player_count = pd.DataFrame({"Total Players" : [len(player_purchases)]})
player_count

,Total Players
0,576


### Summary of Purchase Data

In [4]:
#  ID which types of items were bought
items_bought = raw_csv["Item Name"].value_counts()

#  ID how many different items were bought
items_count = len(items_bought)

#  Average amount spent on a purchase
avg_price = raw_csv["Price"].mean()

#  Get the number of total transactions
trans_count = len(raw_csv["Purchase ID"].unique())

#  Determine the total amount 
total_value = raw_csv["Price"].sum()

#  Summarize findings
purchase_summary = pd.DataFrame({"Number of Unique Items":[items_count], 
                                 "Average Price":["${:,.2f}".format(avg_price)],
                                 "Number of Purchases":[trans_count],
                                 "Total Revenue":["${:,.2f}".format(total_value)]
                                })
purchase_summary

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


### Gender demographics data

In [5]:
#  Get gender breakdowns on purchases
gender_grp = raw_csv.groupby(["Gender"])

#  Get unique ID's within the gender groups
gender_SNs = gender_grp["SN"].unique()

#  Figure out the counts of purchasers for each gender
gender_count = [len(i) for i in gender_SNs]

#  Gather results into a new data frame
gender_data = pd.DataFrame(gender_SNs)
gender_data["Total Count"] = gender_count

#  Add in percentage information
gender_data["Percentage in Decimal"] = gender_data["Total Count"] / gender_data["Total Count"].sum()

#  Formated version of the percentage information
gender_data["Percentage of Players"] = ["{:,.2f}%".format(i * 100) for i in gender_data["Percentage in Decimal"]]

#  Summarize findings
gender_summ = gender_data[["Total Count", "Percentage of Players"]].sort_values(["Total Count"], ascending=False)
gender_summ

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [ ]:
#  Purchase analysis by gender

In [ ]:
#  Age demographics data

In [ ]:
#  Identify top spenders

In [ ]:
#  Identify most popular items

In [ ]:
#  Identify most profitable items